This notebook is specific to the [Openjourney](https://huggingface.co/prompthero/openjourney) model only. Want to use another model? check [here](https://github.com/Lycantant/invoke-ai-gui-colab). <br>

Not familiar with the prompt technique used in InvokeAi? Read [here](https://invoke-ai.github.io/InvokeAI/features/PROMPTS/). And also each model has a different prompt technique. So you should read the links above carefully before starting, so your time will not be wasted.

---
## Requirements
1. Login to ngrok [here](https://dashboard.ngrok.com/login), and create authtoken [here](https://dashboard.ngrok.com/get-started/your-authtoken).
2. Requires 3 Gb disk space on a [Google Drive](https://drive.google.com/drive/my-drive)
3. Don't rush, launch code in each form below one by one.
---
####Visit my Github repo [here](https://github.com/Lycantant/invoke-ai-gui-colab/).
Give a ⭐ if you like it 😁 <br>
Semoga beruntung 🙂

In [ ]:
# @title 01. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title 02. During this stage, Colab crashes, it's normal
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
#@title 03. Installing [InvokeAi](https://github.com/invoke-ai/InvokeAI) { display-mode: "form" }
%cd /home

!git clone -n https://github.com/invoke-ai/InvokeAI.git
%cd InvokeAI
!git checkout tags/v2.2.5

!ln -sf environments-and-requirements/environment-lin-cuda.yml environment.yml
!ls -la

%cd ..

!git clone https://github.com/Lycantant/invoke-ai-gui-colab.git

%cd invoke-ai-gui-colab

%cp cross_attention_control.py ../InvokeAI/ldm/models/diffusion/cross_attention_control.py
%cp globals.py ../InvokeAI/ldm/invoke/globals.py

In [ ]:
#@title 04. Installation normally takes ~10 minutes { display-mode: "form" }
%cd ../InvokeAI
!pip install pyngrok --quiet
!conda env update
!source activate invokeai ; python scripts/configure_invokeai.py --skip-sd-weights --yes

%cp ../invoke-ai-gui-colab/models_yaml/models_openjourney_v4.yaml /root/invokeai/configs/models.yaml

In [ ]:
#@title 05. Downloading Openjourney model (requires 3 Gb disk space on a Google Drive){ display-mode: "form" }
import os

os.system('cd /')
os.system('mkdir -p /root/invokeai/models/ldm/stable-diffusion-v1')

# openjourney v4
if os.path.exists('/content/drive/MyDrive/models/mdjrny-v4.ckpt'):
  print('Model mdjrny-v4.ckpt already exists')
else:
  print('Downloading mdjrny-v4.ckpt')
  os.system('wget -O /content/drive/MyDrive/models/mdjrny-v4.ckpt https://huggingface.co/prompthero/openjourney/resolve/main/mdjrny-v4.ckpt')

# vae
if os.path.exists('/content/drive/MyDrive/models/vae-ft-mse-840000-ema-pruned.ckpt'):
  print('Model vae-ft-mse-840000-ema-pruned.ckpt already exists')
else:
  print('Downloading SD-VAE')
  os.system('wget -O /content/drive/MyDrive/models/vae-ft-mse-840000-ema-pruned.ckpt https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt')

# openjourney v4
!ln -s  /content/drive/MyDrive/models/mdjrny-v4.ckpt /root/invokeai/models/ldm/stable-diffusion-v1/mdjrny-v4.ckpt
!ls -l /root/invokeai/models/ldm/stable-diffusion-v1/mdjrny-v4.ckpt

# vae
!ln -s  /content/drive/MyDrive/models/vae-ft-mse-840000-ema-pruned.ckpt /root/invokeai/models/ldm/stable-diffusion-v1/vae-ft-mse-840000-ema-pruned.ckpt
!ls -l /root/invokeai/models/ldm/stable-diffusion-v1/vae-ft-mse-840000-ema-pruned.ckpt

In [ ]:
#@title 06. Running Invoke AI service { display-mode: "both" }
#@markdown Go to [ngrok](https://dashboard.ngrok.com/get-started/your-authtoken) , and paste your authtoken here.

ngrok_token = "" #@param {type:"string"}
nsfw_checker = 0 #@param {type:"slider", min:0, max:1, step:1}
used_vae = "vae-ft-mse-840000-ema-pruned" #@param ["vae-ft-mse-840000-ema-pruned"]

import os
from pyngrok import ngrok

ngrok.kill()
ngrok.set_auth_token(ngrok_token)
public_url = ngrok.connect(9090).public_url
print(f'Invoke Ai public url: {public_url}')

%cd /home/InvokeAI

model_name = "openjourney_v4"
if used_vae == "vae-ft-mse-840000-ema-pruned":
    model_name = "openjourney_v4"

if nsfw_checker:
    !source activate invokeai ; python scripts/invoke.py --web --model $model_name
else:
    !source activate invokeai ; python scripts/invoke.py --web --no-nsfw_checker --outdir "/content/drive/MyDrive/outputs" --model $model_name

#### Wait until the local url (`http://localhost:9090`) is available.
#### Then click/open public url, it will look something like this ➡ `http://3877-35-204-151-175.ngrok.io`
#### Refresh the page if you see a blank screen
---
Semoga beruntung 😀